In [ ]:
from graph_tool.all import *
import graph_tool.all as gt
import random
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import pandas as pd
from tqdm import tqdm

In [ ]:
Node = 5000

m = 2

times = 100

M_list = [5,10,20,50,100]

beta = 0

max_attempts = 100

In [ ]:
from decimal import Decimal

w_list = [0.50, 0.70, 0.90, 0.95, 0.98]

w_list = [Decimal(str(item)).quantize(Decimal('0.00')) for item in w_list]

In [ ]:
failed_networks = []

for M in tqdm(M_list):
    for W in w_list:
        directory = (f"~/o_t_hayashilab/Network_data/graph-tool/modular_network/N={Node}/m=2/beta={beta}/w={W}/Mo={M}/")
        os.makedirs(os.path.expanduser(directory), exist_ok=True)

        for i in range(times):
            attempts = 0
            G = load_graph(f"~/o_t_hayashilab/Network_data/graph-tool/modular_network/N={Node}/m=2/beta={beta}/w=0.00/Mo={M}/{i}.gt.gz")
            while attempts < max_attempts:
                g = G.copy()
                # 辞書の初期化
                graph_dict = {}
                intra_links_dict = {}
                inter_links_dict = {}

                # inter→intraにリワイヤリングするために、inter, intralinkを識別
                for j in range(1):
                    # wの数だけグラフをコピー
                    graph_dict[f'graph{j}'] = g.copy()

                    # intra, interlinkのリストを初期化
                    intra_link_j = []
                    inter_link_j = []

                    # intra, interlinkを識別
                    for edge in g.edges():
                        source_vertex = edge.source()
                        target_vertex = edge.target()

                        if g.vp.module_number[source_vertex] == g.vp.module_number[target_vertex]:
                            intra_link_j.append(edge)
                        else:
                            inter_link_j.append(edge)

                    intra_links_dict[f'intra_link_{j}'] = intra_link_j
                    inter_links_dict[f'inter_link_{j}'] = inter_link_j

                # Get the list of edges using the key 'intra_link_0'
                intra_link_edges = intra_links_dict['intra_link_0']
                inter_link_edges = inter_links_dict['inter_link_0']

                # Find the number of elements in the list
                num_intra_link_edges = len(intra_link_edges)
                num_inter_link_edges = len(inter_link_edges)

                # Step 1: Calculate R
                R = int(W * num_inter_link_edges)

                # Step 2: Remove R inter_links and store the nodes forming these links in NumPy array
                data = {'Node_ID': [], 'Module_Number': []}

                for _ in range(R):
                    if inter_link_edges:
                        random_inter_link = random.choice(inter_link_edges)
                        source_vertex = random_inter_link.source()
                        target_vertex = random_inter_link.target()

                        # Get the node IDs and module numbers
                        source_node_id = int(g.vp.node_id[source_vertex])
                        target_node_id = int(g.vp.node_id[target_vertex])
                        source_module_number = int(g.vp.module_number[source_vertex])
                        target_module_number = int(g.vp.module_number[target_vertex])

                        # Add nodes forming the link to data dictionary
                        if source_node_id != target_node_id:
                            data['Node_ID'].extend([source_node_id, target_node_id])
                            data['Module_Number'].extend([source_module_number, target_module_number])

                            # Remove the selected inter_link
                            inter_link_edges.remove(random_inter_link)
                            g.remove_edge(random_inter_link)

                # Create NumPy array 'numpy_a' from the data dictionary
                numpy_a = np.column_stack((data['Node_ID'], data['Module_Number']))

                # 新しい NumPy array 'numpy_b' を作成
                numpy_b = np.zeros((0, 2), dtype=int)

                while numpy_a.shape[0] > 0:
                    # 一番上の行を選択
                    selected_row = numpy_a[0]

                    # 選択した行の Node_ID と Module_Number を取得
                    selected_node_id = selected_row[0]
                    selected_module_number = selected_row[1]

                    # 同じ Module_Number に属する行を抽出
                    same_module_rows = numpy_a[numpy_a[:, 1] == selected_module_number]

                    # 異なる Module_Number に属する行を抽出
                    different_module_rows = numpy_a[numpy_a[:, 1] != selected_module_number]

                    # 行と同じ Node_ID である行や、すでに行の Node_ID にリンクを持つ行を除外
                    valid_same_rows = same_module_rows[
                        (same_module_rows[:, 0] != selected_node_id) &
                        (~np.isin(same_module_rows[:, 0], g.get_in_neighbors(g.vertex(selected_node_id))))
                    ]

                    # 行と同じ Node_ID である行や、すでに行の Node_ID にリンクを持つ行を除外
                    valid_different_rows = different_module_rows[
                        (different_module_rows[:, 0] != selected_node_id) &
                        (~np.isin(different_module_rows[:, 0], g.get_in_neighbors(g.vertex(selected_node_id))))
                    ]

                    # リンクをはっていない同じ Module_Number に属するノードが存在する場合
                    if valid_same_rows.shape[0] > 0:
                        # ランダムに行を選択
                        selected_row_j = valid_same_rows[np.random.choice(valid_same_rows.shape[0])]
                        node_a = int(selected_row[0])  # 修正点
                        node_b = int(selected_row_j[0])

                        #　リンクをはる
                        g.add_edge(g.vertex(node_a), g.vertex(node_b))

                        # NumPy array 'numpy_a' から行iと行 j を削除
                        numpy_a = np.delete(numpy_a, [0, np.where(numpy_a[:, 0] == node_b)[0][0]], axis=0)

                    elif valid_different_rows.shape[0] > 0:
                        # ランダムに行を選択
                        selected_row_j = valid_different_rows[np.random.choice(valid_different_rows.shape[0])]
                        node_a = int(selected_row[0])  # 修正点
                        node_b = int(selected_row_j[0])

                        #　リンクをはる
                        g.add_edge(g.vertex(node_a), g.vertex(node_b))

                        # NumPy array 'numpy_a' から行iと行 j を削除
                        numpy_a = np.delete(numpy_a, [0, np.where(numpy_a[:, 0] == node_b)[0][0]], axis=0)

                    else:
                        # 同じ Module_Number に属するノードが存在しない場合
                        # 新たな NumPy array 'numpy_b' に選択した行を追加
                        numpy_b = np.vstack((numpy_b, selected_row))

                        # NumPy array 'numpy_a' から行を削除
                        numpy_a = np.delete(numpy_a, 0, axis=0)

                l = gt.label_largest_component(g)
                u = gt.GraphView(g, vfilt=l)
                sum_u = u.num_vertices()

                if numpy_b.shape[0] == 0 and sum_u == Node:
                    filename = directory + f"{i}.gt.gz"

                    # ネットワークを保存
                    g.save(filename)

                    print(f"{i}個目作成完了")
                    break
                else:
                    # 選択されたノードと同じモジュールに属するノードを集めるdf_c_dict
                    df_c_dict = {}

                    # numpy_bに残ったリンクできていないノードに対して処理を行う
                    for k in range(len(numpy_b)):
                        # ノードIDとモジュール番号を取得
                        c_selected_node_id = numpy_b[k, 0]
                        c_selected_module_number = numpy_b[k, 1]

                        # numpy形式のリストを作るよ
                        df_c_k = np.empty((0, 2), dtype=int)

                        # 全てのノードに対して処理をおこなう
                        for v in g.vertices():
                            node_id_v = int(g.vp.node_id[v])
                            module_number_v = int(g.vp.module_number[v])

                            # Check if the node is not adjacent to the selected node but has the same Module_Number
                            if (node_id_v != c_selected_node_id) and (not g.edge(v, g.vertex(c_selected_node_id))):
                                # Add the node to df_c_i
                                df_c_k = np.vstack((df_c_k, np.array([[node_id_v, module_number_v]], dtype=int)))

                        # Display the information about the selected row and df_c_i
                        #print(f"Selected Node_ID: {c_selected_node_id}, Module_Number: {c_selected_module_number}")
                        #print(f"df_c_{i}:")
                        #print(df_c_i)

                        # Store the DataFrame in the dictionary
                        df_c_dict[f"df_c_{k}"] = df_c_k

                    # 新しいリスト list_b_remove を作成
                    list_b_remove = []

                    for m in range(len(numpy_b) - 1):
                        if m in list_b_remove:
                            continue

                        for n in range(m + 1, len(numpy_b)):
                            if m in list_b_remove or n in list_b_remove:
                                continue

                            row_1, row_2 = numpy_b[m], numpy_b[n]
                            module_number_1, module_number_2 = row_1[1], row_2[1]

                            df_c_o = df_c_dict.get(f"df_c_{m}", np.empty((0, 2), dtype=int))
                            df_c_p = df_c_dict.get(f"df_c_{n}", np.empty((0, 2), dtype=int))

                            if df_c_o.size == 0 or df_c_p.size == 0:
                                continue

                            link_found = False

                            # Operate 3: Search for a link between nodes in df_c_k and df_c_l
                            for q in range(len(df_c_o)):
                                a_node_id = df_c_o[q, 0]

                                for r in range(len(df_c_p)):
                                    b_node_id = df_c_p[r, 0]

                                    if g.edge(g.vertex(int(a_node_id)), g.vertex(int(b_node_id))):
                                        #print(f"異なるモジュール間をつなげているノードはa：{a_node_id}-b：{b_node_id}")
                                        link_found = True
                                        break

                                if link_found:
                                    break

                            # If no link is found in df_c_k and df_c_l, continue to the next iteration
                            if not link_found:
                                continue

                            # Remove the edge between a and b
                            edge_to_remove = g.edge(g.vertex(int(a_node_id)), g.vertex(int(b_node_id)))
                            g.remove_edge(edge_to_remove)

                            g.add_edge(g.vertex(int(row_1[0])), g.vertex(int(a_node_id)))
                            g.add_edge(g.vertex(int(row_2[0])), g.vertex(int(b_node_id)))
                            list_b_remove.extend([m, n])

                            # 新しいリスト list_a_sorted を作成
                            list_a_sorted = []

                            # 対象の Node_ID を取得
                            target_node_ids = [row_1[0], row_2[0], a_node_id, b_node_id]
                            #print(target_node_ids)

                            # 指定された Node_ID を持つ行を探索
                            for idx, row in enumerate(numpy_b):
                                if row[0] in target_node_ids:
                                    list_a_sorted.append(idx)

                            #print(list_a_sorted)

                            # Loop through the selected rows in df_c
                            for s in range(len(list_a_sorted)):
                                # Get the index from list_a_sorted
                                selected_row_idx = list_a_sorted[s]

                                # Get the selected Node_ID and Module_Number from numpy_b using the index
                                c_selected_node_id = numpy_b[selected_row_idx, 0]
                                c_selected_module_number = numpy_b[selected_row_idx, 1]

                                # Get the corresponding df_c_i from df_c_dict using the index
                                df_c_s = df_c_dict.get(f"df_c_{selected_row_idx}")
                                #print(f"更新前：{df_c_i}")
                                # df_c_i を空のNumPy配列で上書き
                                df_c_s = np.empty((0, 2), dtype=int)
                                #print(f"空にした：{df_c_i}")

                                # Traverse all nodes in the graph
                                for x in g.vertices():
                                    node_id_x = int(g.vp.node_id[x])
                                    module_number_x = int(g.vp.module_number[x])

                                    # Check if the node is not adjacent to the selected node but has the same Module_Number
                                    if (node_id_x != c_selected_node_id) and (not g.edge(x, g.vertex(c_selected_node_id))):
                                        if module_number_x == c_selected_module_number:
                                            # Add the node to df_c_i
                                            df_c_s = np.vstack((df_c_s, np.array([[node_id_x, module_number_x]], dtype=int)))

                                #print(f"更新後：{df_c_i}")
                                # Update the DataFrame in the dictionary
                                df_c_dict[f"df_c_{selected_row_idx}"] = df_c_s

                    # 新しい NumPy array 'numpy_b' を作成
                    numpy_b = np.delete(numpy_b, list_b_remove, axis=0)

                    l = gt.label_largest_component(g)
                    u = gt.GraphView(g, vfilt=l)
                    sum_u = u.num_vertices()

                    if numpy_b.shape[0] == 0 and sum_u == Node:

                        filename = directory + f"{i}.gt.gz"

                        # ネットワークを保存
                        g.save(filename)

                        print(f"{i}個目-アナザー作成完了")
                        break
                    else:
                        print(f"{attempts}回目はだめでした")
                        attempts += 1
            else:
                print(f"ネットワークは作成できませんでした:{i}個目")
                failed_networks.append((M, W, i))

In [ ]:
# 失敗したネットワークの情報を出力（またはファイル保存）
if failed_networks:
    print("作成に失敗したネットワーク一覧:")
    for item in failed_networks:
        print(f"M={item[0]}, W={item[1]}, i={item[2]}")